<a href="https://colab.research.google.com/github/MicroprocessorX069/Collaborative-Filtering-for-medical-history/blob/master/CF_dummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls "/content/drive/My Drive"

ls: cannot access '/content/drive/My Drive': No such file or directory


In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

ValueError: ignored

In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1x0SvSkhIDwBBfOkf_cT_T1NIEL2x38a8' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())

title: dummy.csv, id: 1Z3nREEI0X88dJvoXMr3GbRuj0_HNoWBS
downloading to /root/data/dummy.csv
title: ukb8447.001.csv, id: 1l5LpjzqHRrhyOWXErxGhfB17Gt_D1PQf
downloading to /root/data/ukb8447.001.csv


In [0]:
import pandas as pd
df=pd.read_csv("fcodesbucket.csv",header=None)
df=df.iloc[1:,1:]
import math
#adjacent elements of original dataset added here in df_adj
df_adj=pd.DataFrame()
df_adj3=pd.DataFrame()
for x in range(len(df)):
  y=0
  while y+1<len(df.iloc[x]) and not(pd.isna(df.iloc[x,y+1])):
    temp=df.iloc[x,y]
    temp2=df.iloc[x,y+1]
    df2 = pd.DataFrame([[temp,temp2]])
    #df2_rev = pd.DataFrame([[temp2,temp]])
    df_adj=df_adj.append(df2)
    #df_adj=df_adj.append(df2_rev)
    y+=1
#df_adj=df_adj.reset_index() # to make the indices of rows as 0,1,2,3...
df_adj=df_adj.dropna()

36765 rows, 2 columns  (26927 unique combinations; 73.24 %)

Each column has 8.87%, 8.73% of unique codes, respectively.


---



48.761 % times the letter associated with adjacent symptoms are adjcant alphabetically

36.36 % times the letters are same

In [42]:
df_adj=df_adj.dropna()
df_expanded=pd.DataFrame()
df_k=pd.DataFrame()
count=0
for el1,el2 in zip(df_adj[0],df_adj[1]):  
  if abs(ord(el1[0])-ord(el2[0]))<=1 and (el1[1:].isdigit() and el2[1:].isdigit() ):
    df2 = pd.DataFrame([[   ord(el1[0]),int(el1[1:]),ord(el2[0]),int(el2[1:])   ]])
    df_expanded=pd.concat([df_expanded,df2])
    count+=1
  if (ord(el1[0])==75 or ord(el2[0])==75) and (el1[1:].isdigit() and el2[1:].isdigit() ):
    df2 = pd.DataFrame([[   ord(el1[0]),int(el1[1:]),ord(el2[0]),int(el2[1:])   ]])
    df_k=pd.concat([df_k,df2])
df_k
#print(count*100/len(df_adj))

,0,1,2,3
0,76,290,75,625
0,75,625,73,849
0,76,33,75,449
0,75,449,70,99
0,90,38,75,811
0,75,811,75,590
0,75,590,75,317
0,78,950,75,20
0,75,20,75,115
0,75,115,70,99


Symptoms with 'K' have the maximum occurences

8461 Occurences in 37k

In [33]:
df_k.shape

(8461, 4)

In [28]:
import collections
import operator

a=collections.Counter(df_expanded[0])
print(a)

Counter({75: 2897, 70: 2480, 82: 2417, 77: 1477, 84: 1417, 83: 1326, 78: 1194, 73: 1096, 71: 800, 74: 689, 76: 405, 68: 379, 79: 321, 72: 311, 67: 255, 90: 253, 69: 180, 65: 16, 66: 13, 81: 1})


In [31]:
chr(70)

'F'

In [22]:

df_adj[1].drop_duplicates().shape[0]*100/df_adj.shape[0]

8.73657010743914

In [0]:
processedData=pd.get_dummies(df_adj[0])
processedLabel=pd.get_dummies(df_adj[1])
import numpy as np
processedData=np.array(processedData) #direct codes converted to binary freq
processedLabel=np.array(processedLabel)
processedData2=np.array(df_expanded[0])
processedLabel2=np.array(df_expanded[2:])


In [35]:
processedData2

array([[73, '500', 73, '48'],
       [73, '48', 73, '200']], dtype=object)

In [0]:
#@title Parameters for the model { run: "auto" } 

import tensorflow as tf
#run:auto command makes the code block run in real time. Whenever the user amends the input, the block runs automatically for the new set of input.
no_of_layers= 3 #@param {type:"slider", min:0, max:10, step:1}
# A is the list of nodes_in_each_hidden_layer for each of the user defined hidden layers. Eg. A=[128,256,64] implies 3 hidden layers with 128 nodes, 256 nodes and 64 nodes respectively starting from the input layer to the output layer.
A=[]
# No. of nodes in hidden layer 1
nodes_in_layer1 = 5000 #@param {type:"integer"}
A.append(nodes_in_layer1)
#dropout_rate = 0.1 #@param {min:0.000,max:1.000}

nodes_in_layer2 = 5000 #@param {type:"integer"}
A.append(nodes_in_layer2)
nodes_in_layer3 = 5000 #@param {type:"integer"}
A.append(nodes_in_layer3)
validation_splitt=0 #@param{type:"slider",min:0.000,max:1.000,step:0.05}
Activation_function = tf.nn.relu #@param ["relu", "relu6", "crelu", "elu", "selu", "softplus", "softsign", "dropout", "bias_add", "sigmoid", "tanh"]
loss_function = 'kullback_leibler_divergence' #@param ["categorical_crossentropy", "sparse_categorical_crossentropy", "binary_crossentropy","kullback_leibler_divergence","poisson","cosine_proximity"]
optimizer_used = 'SGD' #@param ["SGD", "RMSprop", "Adagrad","Adadelta","Adam","Adamax","Nadam","TFOptimizer"]

In [0]:
#@title Parametres for the model { run: "auto" }

#no_epochs is the integer input for total no. of model parsing -
#from input to generating output and back to updating the weights -
#according to the error for all training examples.

no_epochs=1000 #@param{type:"slider",min:0,max:10000,step:200}
 
import time
start=time.time()

#Libraries for current codeblock
import tensorflow as tf
from tensorflow import nn
nodes_in_output_layer=processedLabel.shape[1]
input_size=processedData.shape[1]

model=tf.keras.models.Sequential()

#Various types of layers in keras: Dense, Flatten, Input, Permute, Reshape.[3]
#Dense implements the operation: output = activation(dot(input, kernel) + bias)[3]

#Adding input layer with size of input i.e. 10 in this case. 
#The first parameter for function Dense() is No. of nodes. in the layer. model.add appends the newly created hidden layer to the model.

model.add(tf.keras.layers.Dense(A[0],activation=Activation_function,input_dim=input_size))
for layer_iterator in range(1,len(A)-1):
  model.add(tf.keras.layers.Dense(A[layer_iterator+1], activation=Activation_function))
#Creating dropout layer
#model.add(tf.keras.layers.Dropout(rate=dropout_rate))
# Creating the output layer with 4 nodes. Since Output layer 
model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))

# Parameters for model creation i.e. optimizers, loss function
                                                                                # metric: Accuracy. Accuracy is the measure of correctness of the model
                                                                                # - , basically rate to which it classifies the inputs correctly.
                                                                                # Validation Accuracy: Percentage of correctly classifying on input chosen from the train dataset itself.
                                                                                # Testing Accuracy:  Percentage of correctly classifying on input from a new dataset following the same problem as the train dataset.

                                                                                # Optimizer: Algorithm to minimize the error/loss by updating weights to improve the training of the model. e.g.
model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
model.summary()
history=model.fit(processedData, processedLabel, epochs= no_epochs)

end=time.time()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 5000)              17125000  
_________________________________________________________________
dense_7 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_8 (Dense)              (None, 3424)              17123424  
Total params: 59,253,424
Trainable params: 59,253,424
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
 1664/74192 [..............................] - ETA: 1:24 - loss: 8.1057 - acc: 0.0012

KeyboardInterrupt: ignored

In [0]:
# loss,acc=model.evaluate(processedData,processedLabel)
# print("Loss=",loss,"\nAccuracy=",acc)
# time_taken=end-start
# print("Time Taken=",end-start)
# %matplotlib inline
# df = pd.DataFrame(history.history)
# df.plot(subplots=True, grid=True, figsize=(10,15))
# print(np.std(df))

In [0]:
predictedTestLabel = []
for i,j in zip(processedData,processedLabel):
    y = model.predict(np.array(i).reshape(-1,5))
    predictedTestLabel.append(y.argmax())
def decodeLabel(encodedLabel):
    if encodedLabel == 0:
        return "a"
    elif encodedLabel == 1:
        return "b"
    elif encodedLabel == 2:
        return "c"
    elif encodedLabel == 3:
        return "d"
    elif encodedLabel == 4:
        return "e"

predictedTestLabel2=[]
for Label in predictedTestLabel:
  predictedTestLabel2.append(decodeLabel(Label))
predictedTestLabel2=pd.DataFrame(predictedTestLabel2)
print(pd.concat([df_adj,predictedTestLabel2],ignore_index=True))

ValueError: ignored

In [0]:
pd.DataFrame(predictedTestLabel2)

,0
0,b
1,a
2,b
3,b
4,b
5,a
6,b
7,b
8,b
9,a
